# Guideline for Prompt engineering (Cont.)

Nuestro objetivo es generar embeddings de un texto para despues poder buscar cosas dentro de este texto por medio de preguntas en lenguaje natural.

## Word embeddings

* Word embeddings o simplemente embeddings son el resultado de un proceso mediante el cual se utiliza alguna tecnica/algoritmo que sea capaz de convertir palabras o texto a vectores de N dimensiones. 

* Estos vectores contienen cierto nivel de información semantica sobre el texto o palabra. 

* Por ejemplo, palabras que son muy similares van a tener valores cercanos en sus representaciones en vectores. 

* Existen diversos modelos que son capaces de hacer un embedding de un texto, en este caso utilizaremos el embedding de OpenAI, el cual tiene la capacidad de posicionas muy bien palabras o textos segun su semantica. 

* Este es el mismo embedding que utiliza GPT3.

* ¿Cómo definir cercanía? --> Métrica Coseno

<img src="Cosine.png" alt="Cosine" width="200" height="150">


In [1]:
# Importamos todas las dependencias requeridas
import gradio as gr # Para desarrollar la interfaz grafica
import openai
import pandas as pd

from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity

# Definimos la API key
openai.api_key  = 'sk-'

### Que es y cómo usar embeddings

Al hacer embedding de un texto/dato, lo convertimos en un vector numérico.

Recordemos que datos similares estarán más cercanos entre si cuando semanticamente son similares.

In [2]:
# Se puede hacer embeeding de palabras o cadenas de texto
palabras = ["casa", "perro", "gato", "lobo", "leon", "zebra", "tigre"]

In [3]:
# Usemos una consulta a Open AI para generar embaddings

diccionario = {}

for i in palabras:
    diccionario[i] = get_embedding( i , engine = "text-embedding-ada-002" )
    # See: https://platform.openai.com/docs/guides/embeddings/what-are-embeddings, for other options
    
#diccionario

RetryError: RetryError[<Future at 0x7fd492e3f7c0 state=finished raised APIError>]

In [ ]:
# Resultado:
diccionario["perro"][:10] # En general la dimensión del vector es 1536

### Operando con embeddings

* Debido a que los embeddings son una representacion vectorial de los datos en un espacio latente, podemos medir la distancia entre dos vectores y asi obtener que tan similares son. 

* Podemos comparar una palabra nueva o alguna de las que ya fueron embebidas y que estan contenidas en el objeto _diccionario_

Obs.: No necesariamente es similitud de objetos. Ejemplo, perro y gato aun siendo "opuestos" están cerca pues tienen una relación.

In [ ]:
# Ejemplo: Comparación

n_palabra = "agujero negro" # Palabra nueva a comparar

palabra_comparar = "perro" # Palabra del diccionario

n_palabra_embed = get_embedding( n_palabra, engine = "text-embedding-ada-002" )

similitud = cosine_similarity( diccionario[palabra_comparar], n_palabra_embed )

print(similitud)

In [ ]:
# Ejemplo: Suma

pd.DataFrame( diccionario["leon"] ) 

In [ ]:
#
sumados = ( pd.DataFrame( diccionario["leon"] ) ) + ( pd.DataFrame( diccionario["zebra"] ) )

sumados

In [ ]:
# ¿A qué se parece más la suma de un león y una zebra?

for key, value in diccionario.items():
    print(key , ":" , cosine_similarity( diccionario[key], sumados ) )

# Procesar datos de un PDF

Haremos ahora un ejemplo donde leemos un PDF para poder hacer preguntas y traer un exctracto del PDF.

In [ ]:
# pip install langchain #pypdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
#
loader = PyPDFLoader("ENIGH2022.pdf")
pages = loader.load_and_split()

In [ ]:
# Un elemento por cada página
pages[3].page_content

In [ ]:
# Objeto que va a hacer los cortes en el texto
split = CharacterTextSplitter( chunk_size = 300, separator = '.\n')

In [ ]:
textos = split.split_documents(pages) # Lista de textos

In [ ]:
print(textos[0].page_content)
#print(textos[0])

In [ ]:
# Extraemos la parte de page_content de cada texto y lo pasamos a un dataframe
textos = [str(i.page_content) for i in textos] #Lista de parrafos
parrafos = pd.DataFrame(textos, columns=["texto"])
parrafos

In [ ]:
# Generamos embedings y guardamos los datos
parrafos['Embedding'] = parrafos["texto"].apply( lambda x: get_embedding( x, engine = 'text-embedding-ada-002' ) ) 
                        # Nueva columna con los embeddings de los parrafos
parrafos.to_csv('ENIGH2022.csv')

In [ ]:
#
parrafos = pd.read_csv('ENIGH2022.csv')
parrafos

In [ ]:
# Función de preguntas y respuestas

def buscar( busqueda, datos, n_resultados = 5):
    busqueda_embed = get_embedding(busqueda, engine = "text-embedding-ada-002")
    datos["Similitud"] = datos['Embedding'].apply(lambda x: cosine_similarity(x, busqueda_embed))
    datos = datos.sort_values("Similitud", ascending=False)
    return datos.iloc[:n_resultados][["texto", "Similitud", "Embedding"]]

In [ ]:
# App
texto_emb = parrafos

with gr.Blocks() as demo:
    busqueda = gr.Textbox(label="Buscar")
    output = gr.DataFrame(headers=['texto'])
    greet_btn = gr.Button("Preguntar")
    greet_btn.click(fn=buscar, inputs=[busqueda, gr.DataFrame(texto_emb)], outputs=output)

demo.launch()


# resp = buscar("Con cuanta vida empiezo?", parrafos, 5) # Reutilizamos la funcion de "buscar" del app de gradio